In [ ]:
# run_env_report.py
# Prints OS, Python, TensorFlow, GPU, CUDA/cuDNN (where available) in one go.

import os
import platform
import subprocess
import sys

def run(cmd):
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
        return out.strip()
    except Exception as e:
        return f"[not available] ({e})"

print("\n=== SYSTEM ===")
print("OS:", platform.platform())
print("Machine:", platform.machine())
print("Processor:", platform.processor())
print("Python:", sys.version.replace("\n", " "))

print("\n=== NVIDIA / CUDA (via nvidia-smi) ===")
print(run(["nvidia-smi"]))

print("\n=== TENSORFLOW ===")
try:
    import tensorflow as tf
    print("TensorFlow version:", tf.__version__)
    print("Built with CUDA:", tf.test.is_built_with_cuda())
    print("GPUs detected:", tf.config.list_physical_devices("GPU"))

    # Build info (often includes CUDA and cuDNN versions for TF builds)
    try:
        bi = tf.sysconfig.get_build_info()
        print("\nTensorFlow build info:")
        for k in sorted(bi.keys()):
            print(f"  {k}: {bi[k]}")
    except Exception as e:
        print("tf.sysconfig.get_build_info() not available:", e)

    # CUDA / cuDNN runtime versions (may be available depending on TF version)
    try:
        from tensorflow.python.platform import build_info as tf_build_info
        print("\nTensorFlow internal build_info (if available):")
        for k in dir(tf_build_info):
            if "cuda" in k.lower() or "cudnn" in k.lower() or "tensorrt" in k.lower():
                print(f"  {k}: {getattr(tf_build_info, k)}")
    except Exception:
        pass

except Exception as e:
    print("TensorFlow not importable:", e)

print("\n=== CUDA TOOLKIT (if installed) ===")
print("nvcc --version:")
print(run(["nvcc", "--version"]))

print("\n=== cuDNN (common Linux paths; may not exist on Windows) ===")
# This is best-effort; on Windows, cuDNN is usually inside CUDA folders or TF wheels.
cudnn_paths = [
    "/usr/include/cudnn_version.h",
    "/usr/local/cuda/include/cudnn_version.h",
]
found = False
for p in cudnn_paths:
    if os.path.exists(p):
        found = True
        print(f"Found: {p}")
        print(run(["bash", "-lc", f"grep -E 'CUDNN_(MAJOR|MINOR|PATCHLEVEL)' {p}"]))
if not found:
    print("[not found in common paths]")

print("\n=== END ===\n")
